Género dataframe
Quitar stopwords
Objetos tipo vectorizer en pandas
Si uso count vectorizer
Precaución: matriz me debe quedar con igual número de columnas

Word to vec ... Vectorizador


No supervisado no necesito etiquetas
En supervisado si


In [1]:
import os
import tarfile
from six.moves import urllib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
# Cargo el dataset

ira_dataset = pd.read_csv("IRA_1.csv", encoding = "ISO-8859-1")
# encoding = "ISO-8859-1" es lo mismo que latin 1

type(ira_dataset)

pandas.core.frame.DataFrame

In [3]:
ira_dataset.shape

(2240, 12)

In [4]:
ira_dataset.index

RangeIndex(start=0, stop=2240, step=1)

In [5]:
# 3 primeras filas del dataframe
ira_dataset.head(3) 

# Se observa:
# Desconfiguracion de texto por caracteres especiales
# algunos campos no vacios pero diligenciados con los simbolos '-' y '_'
# Existe una columna totalmente vacia, incluso sin encabezado (NaN)
# el dataset originalmente contiene 2240 filas y 12 columnas
# columnas criticas objeto del analisis de sentimiento: 'P3.ASPECTOS...' e 'INCIDENTE CRITICO'

,D3.TRANSACCION,CANAL_ATENCION,CAUSA,DESCRIPCION,MOTIVO,MUNICIPIO,D0.SERVICIO,D2.PUNTO DE CONTACTO,Unnamed: 8,D5.LOCALIDAD,P3.ASPECTOS QUE LO SORPRENDIERON O QUE LE MOLESTARON,INCIDENTE CRITICO
0,Revisiones PeriÃ³dicas,_,_,_,_,MEDELLIN,Gas,LÃ­nea de atenciÃ³n,NaN,Metropolitana,-,EXPERIENCIA NORMAL
1,Reclamaciones Presencial (resuelto/Final),PRESENCIAL,INCONFORMIDAD CON EL CONSUMO O PRODUCCION FACT...,INQUILINA RECLAMA POR CONSUMO ALTO DE ENERGIA ...,_,COPACABANA,EnergÃ­a,Oficina de atenciÃ³n,NaN,Metropolitana,PRIMERO ES MUY GRATO SABER QUE HAY FILAS PRIOR...,INCIDENTES NEGATIVOS
2,DaÃ±os EnergÃ­a Prepago,_,_,_,_,_,EnergÃ­a,LÃ­nea de atenciÃ³n,NaN,Regional,_,_


In [6]:
# Renombro columnas criticas para mayor facilidad
ira_dataset = ira_dataset.rename(columns = {'P3.ASPECTOS QUE LO SORPRENDIERON O QUE LE MOLESTARON':'MENSAJE', 
                                            'INCIDENTE CRITICO':'SENTIMIENTO' })

In [7]:
# Descripcion del dataset
ira_dataset.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 12 columns):
D3.TRANSACCION          2240 non-null object
CANAL_ATENCION          2240 non-null object
CAUSA                   2240 non-null object
DESCRIPCION             2240 non-null object
MOTIVO                  2240 non-null object
MUNICIPIO               2240 non-null object
D0.SERVICIO             2240 non-null object
D2.PUNTO DE CONTACTO    2240 non-null object
Unnamed: 8              0 non-null float64
D5.LOCALIDAD            2240 non-null object
MENSAJE                 2240 non-null object
SENTIMIENTO             2240 non-null object
dtypes: float64(1), object(11)
memory usage: 113.8+ KB


In [8]:
# Determino si esta balanceado el dataset validando cantidad de registros por tipo de 'sentimiento'

print("Longitud del dataset", len(ira_dataset))
print()

print("Categorias Sentimientos \n", ira_dataset.loc[:,'SENTIMIENTO'].value_counts())

#balanceado = tweet_dataset['polarity'].value_counts()
#print ("dataset balanceado? \n", balanceado)


Longitud del dataset 2240

Categorias Sentimientos 
 INCIDENTES POSITIVOS    1095
EXPERIENCIA NORMAL       578
INCIDENTES NEGATIVOS     479
_                         88
Name: SENTIMIENTO, dtype: int64


In [9]:
# verifico cuantos Mensajes vacios (sin descripcion textual, solo con '-' o '_')
# estan asociados a Sentimiento Experiencia Normal

#ira_dataset.loc[ira_dataset['MENSAJE'] == '-', ['MENSAJE', 'SENTIMIENTO']]

ira_dataset.loc[(ira_dataset['MENSAJE'] == '-') & (ira_dataset['SENTIMIENTO'] == 'EXPERIENCIA NORMAL'), 
                ['MENSAJE', 'SENTIMIENTO']]

# encontrando que los 578 registros sin mensaje coinciden con los 578 registros de sentimeinto EXPERIENCIA NORMAL, es decir
# que alos refistros sin mensaje los clasificaban con sentimiento EXPERIENCIA NORMAL, algo aparentemente no logico.

,MENSAJE,SENTIMIENTO
0,-,EXPERIENCIA NORMAL
6,-,EXPERIENCIA NORMAL
10,-,EXPERIENCIA NORMAL
12,-,EXPERIENCIA NORMAL
14,-,EXPERIENCIA NORMAL
...,...,...
2225,-,EXPERIENCIA NORMAL
2232,-,EXPERIENCIA NORMAL
2234,-,EXPERIENCIA NORMAL
2236,-,EXPERIENCIA NORMAL


In [10]:
#help(funcion)

In [11]:
# Resumen de los atributos numericos del dataset
ira_dataset.describe()

,Unnamed: 8
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [12]:
# ira_dataset.count()

In [13]:
# Elimino la columna vacia del dataset
#ira_dataset.drop('Unnamed: 8', axis = 1)
del ira_dataset['Unnamed: 8']

In [14]:
 # valido que se haya eliminado la columna unnamed 8
ira_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 11 columns):
D3.TRANSACCION          2240 non-null object
CANAL_ATENCION          2240 non-null object
CAUSA                   2240 non-null object
DESCRIPCION             2240 non-null object
MOTIVO                  2240 non-null object
MUNICIPIO               2240 non-null object
D0.SERVICIO             2240 non-null object
D2.PUNTO DE CONTACTO    2240 non-null object
D5.LOCALIDAD            2240 non-null object
MENSAJE                 2240 non-null object
SENTIMIENTO             2240 non-null object
dtypes: object(11)
memory usage: 96.3+ KB


In [15]:
# Imprimo 5 primero registros con sentimiento se clasifica como INCIDENTE POSITIVO.
ira_dataset[ira_dataset['SENTIMIENTO'] == 'INCIDENTES POSITIVOS'].head()

# Encontramos tambien problemas con caracteres especiales

,D3.TRANSACCION,CANAL_ATENCION,CAUSA,DESCRIPCION,MOTIVO,MUNICIPIO,D0.SERVICIO,D2.PUNTO DE CONTACTO,D5.LOCALIDAD,MENSAJE,SENTIMIENTO
3,DaÃ±os Acueducto,_,_,_,_,MEDELLIN,Acueducto,LÃ­nea de atenciÃ³n,Metropolitana,PORQUE TODO BIEN EN QUE ME RECIBIERON LA LLAMA...,INCIDENTES POSITIVOS
4,Peticiones Gas Oficinas (resuelto),PRESENCIAL,INFORMACION GENERAL,Usuario en calidad de propietario solicita ori...,_,PE?OL,Gas,Oficina de atenciÃ³n,Regional,BUENA ATENCIÃN EN CUANTO A QUE ME ATENDIERON...,INCIDENTES POSITIVOS
9,Revisiones PeriÃ³dicas,_,_,_,_,MEDELLIN,Gas,LÃ­nea de atenciÃ³n,Metropolitana,"ME GUSTÃ MUCHO , NO TUVE NINGÃN PROBLEMA ME ...",INCIDENTES POSITIVOS
16,Reclamaciones Presencial (resuelto/Final),PRESENCIAL,INCONFORMIDAD CON EL CONSUMO O PRODUCCION FACT...,Usuaria en calidad de inquilina reclama por lo...,_,BELLO,Acueducto,Oficina de atenciÃ³n,Metropolitana,CUANDO LLEGUÃ A LA OFICINA ME ATENDIERON DE U...,INCIDENTES POSITIVOS
20,Alternativas de Pago,_,_,_,_,MEDELLIN,No aplica,LÃ­nea de atenciÃ³n,Metropolitana,QUE EL ASESOR ME ATENDIÃ EN LA LÃNEA DE ATEN...,INCIDENTES POSITIVOS


In [16]:
# usando iloc accedamos al valor del segundo registro, de sentimiento positivo en el campo TEXTO y
# calculamos la longitud del mensaje

print(ira_dataset[ira_dataset['SENTIMIENTO'] == 'INCIDENTES POSITIVOS'].MENSAJE.iloc[1])

print("Longitud Mensaje: ", len(ira_dataset[ira_dataset['SENTIMIENTO'] == 'INCIDENTES POSITIVOS'].MENSAJE.iloc[1]))


BUENA ATENCIÃN EN CUANTO A QUE ME  ATENDIERON MUY RÃPIDO Y FUERON MUY ORDENADOS (NO DA MÃS INFORMACION)
Longitud Mensaje:  106


In [17]:
# Imprimo 5 primero registros con sentimiento se clasifica como EXPERIENCIA NORMAL - Neutro.
ira_dataset[ira_dataset['SENTIMIENTO'] == 'EXPERIENCIA NORMAL'].head()

# Vemos mensajes sin descripcion de texto, solo '_' y clasificados como EXPERIENCIA NORMAL

,D3.TRANSACCION,CANAL_ATENCION,CAUSA,DESCRIPCION,MOTIVO,MUNICIPIO,D0.SERVICIO,D2.PUNTO DE CONTACTO,D5.LOCALIDAD,MENSAJE,SENTIMIENTO
0,Revisiones PeriÃ³dicas,_,_,_,_,MEDELLIN,Gas,LÃ­nea de atenciÃ³n,Metropolitana,-,EXPERIENCIA NORMAL
6,Quejas EnergÃ­a (incluido prepago),TELEFONICO,DEMORA EN ATENDER PETICION,NOMBRE: NELSON CASTA?EDA\nCC: 8276620\nCEL: 30...,_,ITAGUI,EnergÃ­a,LÃ­nea de atenciÃ³n,Metropolitana,-,EXPERIENCIA NORMAL
10,Peticiones Gas Oficinas (resuelto),PRESENCIAL,SEPARACION TEMPORAL O PAGO DIRIGIDO,El usuario solicita separaci?n,_,MEDELLIN,Gas,Oficina de atenciÃ³n,Metropolitana,-,EXPERIENCIA NORMAL
12,DaÃ±os Gas,_,_,_,_,MEDELLIN,Gas,LÃ­nea de atenciÃ³n,Metropolitana,-,EXPERIENCIA NORMAL
14,Revisiones PeriÃ³dicas,_,_,_,_,MEDELLIN,Gas,LÃ­nea de atenciÃ³n,Metropolitana,-,EXPERIENCIA NORMAL


In [18]:
# usando iloc accedamos al valor del segundo registro, de sentimiento Neutro en el campo TEXTO y
# calculamos la longitud del mensaje

print(ira_dataset[ira_dataset['SENTIMIENTO'] == 'EXPERIENCIA NORMAL'].MENSAJE.iloc[1])

print("Longitud Mensaje: ", len(ira_dataset[ira_dataset['SENTIMIENTO'] == 'EXPERIENCIA NORMAL'].MENSAJE.iloc[1]))


-
Longitud Mensaje:  1


In [19]:
# Imprimo 5 primero registros con sentimiento se clasifica como INCIDENTE NEGATIVO
ira_dataset[ira_dataset['SENTIMIENTO'] == 'INCIDENTES NEGATIVOS'].head()

,D3.TRANSACCION,CANAL_ATENCION,CAUSA,DESCRIPCION,MOTIVO,MUNICIPIO,D0.SERVICIO,D2.PUNTO DE CONTACTO,D5.LOCALIDAD,MENSAJE,SENTIMIENTO
1,Reclamaciones Presencial (resuelto/Final),PRESENCIAL,INCONFORMIDAD CON EL CONSUMO O PRODUCCION FACT...,INQUILINA RECLAMA POR CONSUMO ALTO DE ENERGIA ...,_,COPACABANA,EnergÃ­a,Oficina de atenciÃ³n,Metropolitana,PRIMERO ES MUY GRATO SABER QUE HAY FILAS PRIOR...,INCIDENTES NEGATIVOS
5,Quejas Gas,TELEFONICO,INCONFORMIDAD TRABAJOS,Nombre: HERNAN DARIO ALVARES GARCES Cedula: 98...,_,SANTO DOMINGO,Gas,LÃ­nea de atenciÃ³n,Regional,LA DEMORA EN EL ARREGLO Y ESTUVE SIN SERVICIO...,INCIDENTES NEGATIVOS
7,Quejas aguas (incluido prepago),TELEFONICO,INCONFORMIDAD PUNTO DE VENTA YO RECAUDO,Nombre: Bernardo De Jes?s Meneses Restrepo\nCc...,_,MEDELLIN,Acueducto,LÃ­nea de atenciÃ³n,Metropolitana,PUES LO QUE ME MOLESTO ES QUE ELLOS LE DICEN A...,INCIDENTES NEGATIVOS
8,Alternativas de Pago,_,_,_,_,SABANETA,No aplica,LÃ­nea de atenciÃ³n,Metropolitana,NO ME PRESTARON EL SERVICIO EN CUANTO A QUE ...,INCIDENTES NEGATIVOS
11,Quejas Gas,TELEFONICO,INCONFORMIDAD CON LA INFORMACION,CLIENTE :EDISON MONSALVE CC :71723720TEL:30030...,_,MEDELLIN,Gas,LÃ­nea de atenciÃ³n,Metropolitana,NO ME RESOLVIERON EL PROBLEMA NO DA MÃS INFOR...,INCIDENTES NEGATIVOS


In [20]:
# usando iloc accedamos al valor del segundo registro, de sentimiento Negativo en el campo TEXTO y
# calculamos la longitud del mensaje

print(ira_dataset[ira_dataset['SENTIMIENTO'] == 'INCIDENTES NEGATIVOS'].MENSAJE.iloc[1])

print("Longitud Mensaje: ", len(ira_dataset[ira_dataset['SENTIMIENTO'] == 'INCIDENTES NEGATIVOS'].MENSAJE.iloc[1]))


LA DEMORA EN EL ARREGLO Y ESTUVE  SIN SERVICIO  POR VARIOS DÃAS Y NO HABÃA RESPUESTA NI POR PARTE DE EPM NI DEL CONTRATISTA ME PONÃAN DE UN LADO A OTRO
Longitud Mensaje:  154


In [ ]:
#help(round)


In [20]:
# valido existencia de valores nulos
#print(ira_dataset.isnull()) 

      D3.TRANSACCION  CANAL_ATENCION  CAUSA  DESCRIPCION  MOTIVO  MUNICIPIO  \
0              False           False  False        False   False      False   
1              False           False  False        False   False      False   
2              False           False  False        False   False      False   
3              False           False  False        False   False      False   
4              False           False  False        False   False      False   
...              ...             ...    ...          ...     ...        ...   
2235           False           False  False        False   False      False   
2236           False           False  False        False   False      False   
2237           False           False  False        False   False      False   
2238           False           False  False        False   False      False   
2239           False           False  False        False   False      False   

      D0.SERVICIO  D2.PUNTO DE CONTACTO  D5.LOCALID

In [ ]:
# valida si hay filas con nan y en caso de que si usa el metodo .dropna(), ajusta el parametro inplace en True.
#print(np.isnan(ira_dataset))

#x = x[numpy.logical_not(numpy.isnan(x))]
#print(ira_dataset['Site Fill'].value_counts(dropna=False))

In [21]:
# Determino si existen celdas nulas por columnas (campos)

def num_missing(x):
    return sum(x.isnull())

# nulos por columna
colnull = ira_dataset.apply(num_missing, axis = 0)
print("Nulos por columna: \n" , colnull)


# sin embargo toma el simbolo "-" y "_" como si no fuera nulo

Nulos por columna: 
 D3.TRANSACCION          0
CANAL_ATENCION          0
CAUSA                   0
DESCRIPCION             0
MOTIVO                  0
MUNICIPIO               0
D0.SERVICIO             0
D2.PUNTO DE CONTACTO    0
D5.LOCALIDAD            0
MENSAJE                 0
SENTIMIENTO             0
dtype: int64


In [22]:
# Determino si existen celdas nulas por filas
filnull = ira_dataset.apply(num_missing, axis = 1)

# Por fila
print ("Nulos por fila: \n", filnull.head())

Nulos por fila: 
 0    0
1    0
2    0
3    0
4    0
dtype: int64


In [36]:
# Convierto el simbolo "-" y el "_" a nulo - missing value

ira_dataset_mod = ira_dataset.replace(to_replace = ["-", "_"], value = "NA")
#ira_dataset_mod = ira_dataset.replace(to_replace = "-", value = "NaN")

ira_dataset_mod.head()

,D3.TRANSACCION,CANAL_ATENCION,CAUSA,DESCRIPCION,MOTIVO,MUNICIPIO,D0.SERVICIO,D2.PUNTO DE CONTACTO,D5.LOCALIDAD,MENSAJE,SENTIMIENTO
0,Revisiones PeriÃ³dicas,NA,NA,NA,NA,MEDELLIN,Gas,LÃ­nea de atenciÃ³n,Metropolitana,NA,EXPERIENCIA NORMAL
1,Reclamaciones Presencial (resuelto/Final),PRESENCIAL,INCONFORMIDAD CON EL CONSUMO O PRODUCCION FACT...,INQUILINA RECLAMA POR CONSUMO ALTO DE ENERGIA ...,NA,COPACABANA,EnergÃ­a,Oficina de atenciÃ³n,Metropolitana,PRIMERO ES MUY GRATO SABER QUE HAY FILAS PRIOR...,INCIDENTES NEGATIVOS
2,DaÃ±os EnergÃ­a Prepago,NA,NA,NA,NA,NA,EnergÃ­a,LÃ­nea de atenciÃ³n,Regional,NA,NA
3,DaÃ±os Acueducto,NA,NA,NA,NA,MEDELLIN,Acueducto,LÃ­nea de atenciÃ³n,Metropolitana,PORQUE TODO BIEN EN QUE ME RECIBIERON LA LLAMA...,INCIDENTES POSITIVOS
4,Peticiones Gas Oficinas (resuelto),PRESENCIAL,INFORMACION GENERAL,Usuario en calidad de propietario solicita ori...,NA,PE?OL,Gas,Oficina de atenciÃ³n,Regional,BUENA ATENCIÃN EN CUANTO A QUE ME ATENDIERON...,INCIDENTES POSITIVOS


In [37]:
#En primer lugar, lo que deberíamos hacer es controlar si existen valores faltantes o nulos; 
#esto lo podemos realizar utilizando el método isnull() del siguiente modo:

# Controlando valores nulos
#ira_dataset_mod.isnull().any().any()

ira_dataset_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 11 columns):
D3.TRANSACCION          2240 non-null object
CANAL_ATENCION          2240 non-null object
CAUSA                   2240 non-null object
DESCRIPCION             2240 non-null object
MOTIVO                  2240 non-null object
MUNICIPIO               2240 non-null object
D0.SERVICIO             2240 non-null object
D2.PUNTO DE CONTACTO    2240 non-null object
D5.LOCALIDAD            2240 non-null object
MENSAJE                 2240 non-null object
SENTIMIENTO             2240 non-null object
dtypes: object(11)
memory usage: 96.3+ KB


In [28]:

def num_missing(x):
    return sum(x.isnull())

# nulos por columna
colnull = ira_dataset_mod.apply(num_missing, axis = 0)
print("Nulos por columna: \n" , colnull)

# o:
#ira_dataset_mod.isnull().sum()
#ira_dataset_mod.isnull().sum().sum()

Nulos por columna: 
 D3.TRANSACCION          0
CANAL_ATENCION          0
CAUSA                   0
DESCRIPCION             0
MOTIVO                  0
MUNICIPIO               0
D0.SERVICIO             0
D2.PUNTO DE CONTACTO    0
D5.LOCALIDAD            0
MENSAJE                 0
SENTIMIENTO             0
dtype: int64


In [ ]:
ira_dataset_mod.head()

In [ ]:
# Cuales son los descriptores que nos aportan informacion para el analisis de sentimientos?, cuales no?
# usando el metodo drop() de los DataFrame, elimina los campos que no son necesarios para el analisis 
# de sentimientos, recuerda ajustar un atributo en 1 y otro en True.
#ira_dataset.columns 
ira_dataset.drop(['D3.TRANSACCION','CANAL_ATENCION','CAUSA','DESCRIPCION', 'MOTIVO', 'MUNICIPIO',
'D0.SERVICIO', 'D2.PUNTO DE CONTACTO', 'D5.LOCALIDAD'], axis = 1, inplace = True)
# axis = 1 elimina valores de la columna     


In [ ]:
ira_dataset.head()